In [ ]:
%pip install -qU langchain-community langchain langchain-groq
!wget https://github.com/FelipeAugus/agente-com-langchain-ipynb/raw/main/desenvolvimento-de-testes-unitarios/prompt.md -O prompt.md

import os
import requests
from google.colab import userdata
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_groq import ChatGroq

groq_api = userdata.get("GROQ_API_KEY") # https://console.groq.com/keys
os.environ["GROQ_API_KEY"] = groq_api


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.4/131.4 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.3 MB/s eta 0:00:00


In [19]:
response = requests.get("https://api.groq.com/openai/v1/models", headers={"Authorization": f"Bearer {groq_api}"})
data = response.json()
print("Modelos disponiveis do Groq")
for model in data.get("data", []): print(model["id"])


Modelos disponiveis do Groq
allam-2-7b
meta-llama/llama-prompt-guard-2-22m
openai/gpt-oss-20b
llama-3.1-8b-instant
distil-whisper-large-v3-en
meta-llama/llama-4-scout-17b-16e-instruct
meta-llama/llama-prompt-guard-2-86m
compound-beta-mini
openai/gpt-oss-120b
llama3-70b-8192
gemma2-9b-it
playai-tts-arabic
playai-tts
meta-llama/llama-guard-4-12b
whisper-large-v3
llama3-8b-8192
llama-3.3-70b-versatile
deepseek-r1-distill-llama-70b
meta-llama/llama-4-maverick-17b-128e-instruct
qwen/qwen3-32b
moonshotai/kimi-k2-instruct
compound-beta
whisper-large-v3-turbo


In [20]:
class UnitTestsDeveloperAgent():
    def __init__(self):
        model_id = "llama-3.1-8b-instant"
        self.llm = ChatGroq(model=model_id, temperature=0)
        self.system_msg = ""
        with open("prompt.md", "r") as f: self.system_msg = f.read()

    def generate_tests(self, codigo_fonte: str) -> str:
        threats = self.llm.invoke([
            SystemMessage(content=self.system_msg),
            HumanMessage(content=codigo_fonte)
        ])
        return threats

agente = UnitTestsDeveloperAgent()

In [21]:
testes = agente.generate_tests(
    codigo_fonte="""
    def is_par(numero):
        return numero % 2 == 0
    """,
)
print(testes.content)

```python
import pytest

def test_dado_numero_quando_par_entao_retorna_true():
    # Arrange
    numero = 2

    # Act
    resultado = is_par(numero)

    # Assert
    assert resultado is True


def test_dado_numero_quando_impar_entao_retorna_false():
    # Arrange
    numero = 3

    # Act
    resultado = is_par(numero)

    # Assert
    assert resultado is False


def test_dado_numero_zero_entao_retorna_true():
    # Arrange
    numero = 0

    # Act
    resultado = is_par(numero)

    # Assert
    assert resultado is True


def test_dado_numero_negativo_entao_retorna_true():
    # Arrange
    numero = -4

    # Act
    resultado = is_par(numero)

    # Assert
    assert resultado is True


def test_dado_tipo_nao_numerico_entao_levanta_typeerror():
    # Arrange
    numero = "a"

    # Act / Assert
    with pytest.raises(TypeError):
        is_par(numero)


def test_dado_tipo_complexo_entao_levanta_typeerror():
    # Arrange
    numero = 1 + 2j

    # Act / Assert
    with pytest.rai

In [22]:
testes = agente.generate_tests(
    codigo_fonte="""
    def normalizar_nome(nome: str) -> str:
        if nome is None:
            raise ValueError("nome não pode ser None")
        nome = nome.strip()
        nome = re.sub(r"\s+", " ", nome)
        if nome == "":
            return ""
        return nome.title()

    def gerar_id_pedido(prefixo: str, seq_provider) -> str:
        if not isinstance(prefixo, str) or not re.fullmatch(r"[A-Z]{2,5}", prefixo):
            raise ValueError("prefixo inválido (use 2-5 letras maiúsculas)")
        seq = seq_provider()
        if not isinstance(seq, int) or seq < 1:
            raise ValueError("sequência inválida do provedor (>= 1)")
        return f"{prefixo}-{seq:06d}"
    """,
)
print(testes.content)

```python
import pytest
import re
from unittest.mock import Mock

def test_dado_nome_nulo_quando_normalizar_nome_entao_levanta_value_error():
    # Arrange
    nome = None

    # Act / Assert
    with pytest.raises(ValueError):
        normalizar_nome(nome)


def test_dado_nome_vazio_quando_normalizar_nome_entao_retorna_string_vazia():
    # Arrange
    nome = ""

    # Act
    resultado = normalizar_nome(nome)

    # Assert
    assert resultado == ""


def test_dado_nome_com_multiplos espaços_quando_normalizar_nome_entao_retorna_nome_sem espaços():
    # Arrange
    nome = "   João   Silva   "

    # Act
    resultado = normalizar_nome(nome)

    # Assert
    assert resultado == "João Silva"


def test_dado_nome_com_caracteres_especiais_quando_normalizar_nome_entao_retorna_nome_sem_caracteres_especiais():
    # Arrange
    nome = "João# Silva$"

    # Act
    resultado = normalizar_nome(nome)

    # Assert
    assert resultado == "João Silva"


def test_dado_nome_com_letras_minusculas